In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from random import random
from random import seed
from math import exp

In [10]:
df=pd.read_csv('dataset.csv',index_col=0)

In [11]:
df.head()

,trial1,trial2,trial3,trial4,trial5,label
0,0.174107,-0.195709,-2.181306,1.085189,1.127734,1
1,1.072546,-1.350565,-0.647296,-1.003272,0.286490,1
2,-1.661356,-0.400861,-1.292950,0.351150,0.809441,1
3,-0.051537,1.185729,1.479820,0.343484,-1.280368,1
4,1.400416,0.011803,1.126713,-0.652088,0.962291,1


In [12]:
df.describe()

,trial1,trial2,trial3,trial4,trial5,label
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,-0.012211,-0.020811,-0.037187,0.000582,0.083383,0.555000
std,1.013218,1.408401,1.312847,1.330810,0.993630,0.498213
min,-2.549133,-3.910150,-4.058890,-3.970649,-3.410254,0.000000
25%,-0.634045,-1.377796,-0.955089,-1.021369,-0.546093,0.000000
50%,-0.047067,0.046403,0.150157,0.327429,0.103438,1.000000
75%,0.665015,1.188999,0.940074,1.027700,0.717600,1.000000
max,2.576821,2.825650,2.664412,2.788147,2.261946,1.000000


In [16]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    network=list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [25]:
def activate(weights, inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation+=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [30]:
def forward_propagate(network,raw):
    inputs=raw
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'], inputs)
            neuron['output']=transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs


def transfer_derivative(output):
    return output * (1.0 - output)

In [36]:
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [42]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs=row[:-1]
        if i!=0:
            inputs=[neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1]+=l_rate*neuron['delta']

In [45]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>Epoch=%d, Learning_rate=%.3f, Error=%.3f' % (epoch, l_rate, sum_error))

In [47]:
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [49]:

X=np.array(df.loc[:,df.columns != 'label'])
y=np.array(df.loc[:,df.columns == 'label'])

In [62]:
dataset=np.array(df[:])
dataset

array([[ 0.17410676, -0.19570903, -2.18130625,  1.08518927,  1.12773397,
         1.        ],
       [ 1.07254601, -1.35056543, -0.647296  , -1.00327157,  0.28648957,
         1.        ],
       [-1.66135607, -0.4008605 , -1.29295026,  0.35114996,  0.80944116,
         1.        ],
       ...,
       [ 0.3299558 ,  1.27467778,  1.50805919,  0.41804312, -1.65957122,
         1.        ],
       [ 0.61177012,  0.0577717 , -1.49430942,  0.94012413, -0.91271051,
         0.        ],
       [-1.81544894, -1.5662738 ,  0.63642427, -1.9812005 ,  0.22954838,
         1.        ]])

In [55]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
print(n_inputs,n_outputs)

5 2


In [57]:
train_dataset=dataset[:150]
test_dataset=dataset[150:]


In [58]:
network=initialize_network(n_inputs,1,n_outputs)
train_network(network, train_dataset, 0.5, 20, n_outputs)

>Epoch=0, Learning_rate=0.500, Error=76.441
>Epoch=1, Learning_rate=0.500, Error=68.880
>Epoch=2, Learning_rate=0.500, Error=58.296
>Epoch=3, Learning_rate=0.500, Error=52.370
>Epoch=4, Learning_rate=0.500, Error=50.313
>Epoch=5, Learning_rate=0.500, Error=49.484
>Epoch=6, Learning_rate=0.500, Error=49.065
>Epoch=7, Learning_rate=0.500, Error=48.833
>Epoch=8, Learning_rate=0.500, Error=48.701
>Epoch=9, Learning_rate=0.500, Error=48.617
>Epoch=10, Learning_rate=0.500, Error=48.551
>Epoch=11, Learning_rate=0.500, Error=48.492
>Epoch=12, Learning_rate=0.500, Error=48.433
>Epoch=13, Learning_rate=0.500, Error=48.371
>Epoch=14, Learning_rate=0.500, Error=48.307
>Epoch=15, Learning_rate=0.500, Error=48.239
>Epoch=16, Learning_rate=0.500, Error=48.169
>Epoch=17, Learning_rate=0.500, Error=48.099
>Epoch=18, Learning_rate=0.500, Error=48.028
>Epoch=19, Learning_rate=0.500, Error=47.958


In [63]:
for layer in network:
    print(layer)

[{'weights': [-0.8791516881585492, 1.5424882327436331, 5.0505707619071085, -1.2269563125227407, 0.26123210517493295, -2.000347192671955], 'output': 0.00011910146237192811, 'delta': -2.6438777385447488e-05}]
[{'weights': [-3.3078330514395953, 1.3692399821897387], 'output': 0.7944670855078335, 'delta': 0.033561291935016335}, {'weights': [3.309392092532545, -1.3696915568390988], 'output': 0.205457466548289, 'delta': -0.03353984166510883}]


In [65]:
y_test=[]
y_pred=[]
for row in test_dataset:
    prediction = predict(network, row)
    y_test.append(row[-1])
    y_pred.append(prediction)